In [ ]:
# import libraries 
import pandas as pd
import numpy as np
import warnings 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
from string import ascii_letters
import plotly.express as px
import sys
import gc
import pickle
import time


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from sklearn.pipeline import Pipeline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU
from keras.utils import np_utils

# algorithmic library

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer

from xgboost import XGBRegressor
from xgboost import plot_importance

In [ ]:
# Load all the data sets 

df_train = pd.read_csv('data/sales_train.csv')
df_test = pd.read_csv('data/test.csv')
df_submission = pd.read_csv('data/sample_submission.csv')
df_shop = pd.read_csv('data/shops.csv')
df_item_cat = pd.read_csv('data/item_categories.csv')
df_item = pd.read_csv('data/items.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.describe(include='all').T

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], format='%d.%m.%Y')

In [ ]:
df_train.describe().T

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.nunique()

In [ ]:
df_train.head()

In [ ]:
sns.boxplot(x=df_train.item_cnt_day)

In [ ]:
sns.boxplot(x=df_train.item_price)

In [ ]:
df_train = df_train[(df_train.item_price < 300000 )& (df_train.item_cnt_day < 1000)]

In [ ]:
df_train = df_train[df_train.item_price > 0].reset_index(drop = True)
df_train.loc[df_train.item_cnt_day < 1, "item_cnt_day"] = 0

In [ ]:
sns.boxplot(x=df_train.item_price)

In [ ]:
sns.boxplot(x=df_train.item_cnt_day)

In [ ]:

df_train.loc[df_train.shop_id == 0, 'shop_id'] = 57
df_test.loc[df_test.shop_id == 0, 'shop_id'] = 57

df_train.loc[df_train.shop_id == 1, 'shop_id'] = 58
df_test.loc[df_test.shop_id == 1, 'shop_id'] = 58

df_train.loc[df_train.shop_id == 10, 'shop_id'] = 11
df_test.loc[df_test.shop_id == 10, 'shop_id'] = 11

In [ ]:
df_shop.loc[ df_shop.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
df_shop["city"] = df_shop.shop_name.str.split(" ").map( lambda x: x[0] )
df_shop["category"] = df_shop.shop_name.str.split(" ").map( lambda x: x[1] )
df_shop.loc[df_shop.city == "!Якутск", "city"] = "Якутск"

In [ ]:
category = []
for cat in df_shop.category.unique():
    if len(df_shop[df_shop.category == cat]) >= 5:
        category.append(cat)
df_shop.category = df_shop.category.apply( lambda x: x if (x in category) else "other" )


In [ ]:
from sklearn.preprocessing import LabelEncoder
df_shop["shop_category"] = LabelEncoder().fit_transform( df_shop.category )
df_shop["shop_city"] = LabelEncoder().fit_transform( df_shop.city )
df_shop = df_shop[["shop_id", "shop_category", "shop_city"]]

In [ ]:
df_item_cat["type_code"] = df_item_cat.item_category_name.apply( lambda x: x.split(" ")[0] ).astype(str)
df_item_cat.loc[ (df_item_cat.type_code == "Игровые")| (df_item_cat.type_code == "Аксессуары"), "category" ] = "Игры"

In [ ]:
category = []
for cat in df_item_cat.type_code.unique():
    if len(df_item_cat[df_item_cat.type_code == cat]) >= 5: 
        category.append( cat )
df_item_cat.type_code = df_item_cat.type_code.apply(lambda x: x if (x in category) else "etc")

In [ ]:
df_item_cat.type_code = LabelEncoder().fit_transform(df_item_cat.type_code)
df_item_cat["split"] = df_item_cat.item_category_name.apply(lambda x: x.split("-"))
df_item_cat["subtype"] = df_item_cat.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
df_item_cat["subtype_code"] = LabelEncoder().fit_transform( df_item_cat["subtype"] )
df_item_cat = df_item_cat[["item_category_id", "subtype_code", "type_code"]]

In [ ]:
import re
def name_correction(x):
    x = x.lower() # all letters lower case
    x = x.partition('[')[0] # partition by square brackets
    x = x.partition('(')[0] # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # remove special characters
    x = x.replace('  ', ' ') # replace double spaces with single spaces
    x = x.strip() # remove leading and trailing white space
    return x

In [ ]:
df_item["name1"], df_item["name2"] = df_item.item_name.str.split("[", 1).str
df_item["name1"], df_item["name3"] = df_item.item_name.str.split("(", 1).str


df_item["name2"] = df_item.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
df_item["name3"] = df_item.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# fill nulls with '0'
df_item = df_item.fillna('0')

df_item["item_name"] = df_item["item_name"].apply(lambda x: name_correction(x))

# return all characters except the last if name 2 is not "0" - the closing bracket
df_item.name2 = df_item.name2.apply( lambda x: x[:-1] if x !="0" else "0")


In [ ]:
df_item["type"] = df_item.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
df_item.loc[(df_item.type == "x360") | (df_item.type == "xbox360") | (df_item.type == "xbox 360") ,"type"] = "xbox 360"
df_item.loc[ df_item.type == "", "type"] = "mac"
df_item.type = df_item.type.apply( lambda x: x.replace(" ", "") )
df_item.loc[ (df_item.type == 'pc' )| (df_item.type == 'pс') | (df_item.type == "pc"), "type" ] = "pc"
df_item.loc[ df_item.type == 'рs3' , "type"] = "ps3"

In [ ]:
group_sum = df_item.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] <40:
        drop_cols.append(cat)
df_item.name2 = df_item.name2.apply( lambda x: "other" if (x in drop_cols) else x )
df_item = df_item.drop(["type"], axis = 1)

In [ ]:
df_item.name2 = LabelEncoder().fit_transform(df_item.name2)
df_item.name3 = LabelEncoder().fit_transform(df_item.name3)

df_item.drop(["item_name", "name1"],axis = 1, inplace= True)
df_item.head()

In [ ]:
from itertools import product
import time
ts = time.time()
matrix = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales = df_train[df_train.date_block_num == i]
    matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )

matrix = pd.DataFrame( np.vstack(matrix), columns = cols )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values( cols, inplace = True )
time.time()- ts

In [ ]:
df_train["revenue"] = df_train["item_cnt_day"] * df_train["item_price"]

In [ ]:

group = df_train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"]
group.reset_index( inplace = True)
matrix = pd.merge( matrix, group, on = cols, how = "left" )
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)


In [ ]:
df_test["date_block_num"] = 34
df_test["date_block_num"] = df_test["date_block_num"].astype(np.int8)
df_test["shop_id"] = df_test.shop_id.astype(np.int8)
df_test["item_id"] = df_test.item_id.astype(np.int16)

In [ ]:


matrix = pd.concat([matrix, df_test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=cols)
matrix.fillna( 0, inplace = True )


In [ ]:

matrix = pd.merge( matrix, df_shop, on = ["shop_id"], how = "left" )
matrix = pd.merge(matrix, df_item, on = ["item_id"], how = "left")
matrix = pd.merge( matrix, df_item_cat, on = ["item_category_id"], how = "left" )
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["subtype_code"] = matrix["subtype_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)


In [ ]:
def lag_feature( df,lags, cols ):
    for col in cols:
        print(col)
        tmp = df[["date_block_num", "shop_id","item_id",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
matrix = lag_feature( matrix, [1,2,3], ["item_cnt_month"] )

In [ ]:

group = matrix.groupby( ["date_block_num"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num"], how = "left")
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1], ["date_avg_item_cnt"] )
matrix.drop( ["date_avg_item_cnt"], axis = 1, inplace = True )


In [ ]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix.date_item_avg_item_cnt = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3], ['date_item_avg_item_cnt'])
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

In [ ]:

group = matrix.groupby( ["date_block_num","shop_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_avg_item_cnt"] )
matrix.drop( ["date_shop_avg_item_cnt"], axis = 1, inplace = True )



In [ ]:

group = matrix.groupby( ["date_block_num","shop_id","item_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_item_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id","item_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_item_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_item_avg_item_cnt"] )
matrix.drop( ["date_shop_item_avg_item_cnt"], axis = 1, inplace = True )


In [ ]:

group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix.date_shop_subtype_avg_item_cnt = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_shop_subtype_avg_item_cnt'])
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)


In [ ]:

group = matrix.groupby(['date_block_num', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_city_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', "shop_city"], how='left')
matrix.date_city_avg_item_cnt = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_city_avg_item_cnt'])
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)


In [ ]:

group = matrix.groupby(['date_block_num', 'item_id', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'shop_city'], how='left')
matrix.date_item_city_avg_item_cnt = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_item_city_avg_item_cnt'])


In [ ]:

group = df_train.groupby( ["item_id"] ).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge( group, on = ["item_id"], how = "left" )
matrix["item_avg_item_price"] = matrix.item_avg_item_price.astype(np.float16)


group = df_train.groupby( ["date_block_num","item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge(group, on = ["date_block_num","item_id"], how = "left")
matrix["date_item_avg_item_price"] = matrix.date_item_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
matrix = lag_feature( matrix, lags, ["date_item_avg_item_price"] )
for i in lags:
    matrix["delta_price_lag_" + str(i) ] = (matrix["date_item_avg_item_price_lag_" + str(i)]- matrix["item_avg_item_price"] )/ matrix["item_avg_item_price"]

def select_trends(row) :
    for i in lags:
        if row["delta_price_lag_" + str(i)]:
            return row["delta_price_lag_" + str(i)]
    return 0

matrix["delta_price_lag"] = matrix.apply(select_trends, axis = 1)
matrix["delta_price_lag"] = matrix.delta_price_lag.astype( np.float16 )
matrix["delta_price_lag"].fillna( 0 ,inplace = True)

features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]
for i in lags:
    features_to_drop.append("date_item_avg_item_price_lag_" + str(i) )
    features_to_drop.append("delta_price_lag_" + str(i) )
matrix.drop(features_to_drop, axis = 1, inplace = True)


In [ ]:

group = df_train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["date_shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(["shop_id"]).agg({ "date_block_num":["mean"] })
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace = True )

matrix = matrix.merge( group, on = ["shop_id"], how = "left" )
matrix["shop_avg_revenue"] = matrix.shop_avg_revenue.astype(np.float32)
matrix["delta_revenue"] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix["delta_revenue"] = matrix["delta_revenue"]. astype(np.float32)

matrix = lag_feature(matrix, [1], ["delta_revenue"])
matrix["delta_revenue_lag_1"] = matrix["delta_revenue_lag_1"].astype(np.float32)
matrix.drop( ["date_shop_revenue", "shop_avg_revenue", "delta_revenue"] ,axis = 1, inplace = True)


In [ ]:
matrix["month"] = matrix["date_block_num"] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix["days"] = matrix["month"].map(days).astype(np.int8)

In [ ]:

matrix["item_shop_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')
matrix["item_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id"])["date_block_num"].transform('min')


In [ ]:

matrix = matrix[matrix["date_block_num"] > 3]


In [ ]:
matrix.head().T

In [ ]:
matrix.to_pickle('data2.pkl')

In [ ]:
data = matrix.copy()
del matrix
gc.collect()

In [ ]:
data[data["date_block_num"]==34].shape

In [ ]:
data = pd.read_pickle('demo/data/data.pkl')

In [ ]:
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    # 'date_shop_type_avg_item_cnt_lag_1',
    # 'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    # 'date_type_avg_item_cnt_lag_1',
    # 'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    # 'delta_revenue_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
del data
gc.collect();

In [ ]:
# import optuna
# from sklearn.metrics import mean_squared_error

In [ ]:
# def objective(trial, X_train, Y_train, X_valid, Y_valid):
    
#     param = {
#         'tree_method':'gpu_hist',  # Use GPU acceleration
#         'lambda': trial.suggest_loguniform(
#             'lambda', 1e-3, 10.0
#         ),
#         'alpha': trial.suggest_loguniform(
#             'alpha', 1e-3, 10.0
#         ),
#         'colsample_bytree': trial.suggest_categorical(
#             'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
#         ),
#         'subsample': trial.suggest_categorical(
#             'subsample', [0.6,0.7,0.8,1.0]
#         ),
#         'learning_rate': trial.suggest_categorical(
#             'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
#         ),
#         'n_estimators': trial.suggest_categorical(
#             "n_estimators", [150, 200, 300, 3000]
#         ),
#         'max_depth': trial.suggest_categorical(
#             'max_depth', [4,5,7,9,11,13,15,17]
#         ),
#         'random_state': 42,
#         'min_child_weight': trial.suggest_int(
#             'min_child_weight', 1, 300
#         ),
#     }
#     model = XGBRegressor(**param)  
    
#     model.fit(
#         X_train, 
#         Y_train, 
#         eval_metric="rmse", 
#         eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
#         verbose=True, 
#         early_stopping_rounds = 100)
    
#     preds = model.predict(X_valid)
    
#     rmse = mean_squared_error(Y_valid, preds,squared=False)
    
#     return rmse

In [ ]:
# study = optuna.create_study(direction='minimize')
# # study.optimize(objective, n_trials=5)
# study.optimize(lambda trial : objective(trial, X_train, Y_train, X_valid, Y_valid), n_trials=5)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [ ]:
# finally plot best parameters
# study.best_params

In [ ]:
# best_params = study.best_params
# best_params['tree_method'] = 'gpu_hist'
# best_params['random_state'] = 42

# clf = XGBRegressor(**(best_params))

# clf.fit(X_train, Y_train)

In [ ]:
# from joblib import dump, load
# dump(clf, 'xgb_optuna.joblib') 

In [ ]:
# Y_pred = clf.predict(X_valid).clip(0, 20)
# Y_test = clf.predict(X_test).clip(0, 20)

# submission = pd.DataFrame({
#     "ID": df_test.index, 
#     "item_cnt_month": Y_test
# })
# submission.to_csv('submission.csv', index=False)

In [ ]:
# from xgboost import plot_importance

# def plot_features(booster, figsize):    
#     fig, ax = plt.subplots(1,1,figsize=figsize)
#     return plot_importance(booster=booster, ax=ax)

# plot_features(clf, (10,14))

In [ ]:


from xgboost import XGBRegressor
model = XGBRegressor(
    max_depth=6,
    n_estimators=1000,
    min_child_weight=26, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,
    alpha=0.170,
    tree_method='gpu_hist',
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 20)



In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)

In [ ]:
# from xgboost import plot_importance

# def plot_features(booster, figsize):    
#     fig, ax = plt.subplots(1,1,figsize=figsize)
#     return plot_importance(booster=booster, ax=ax)

# plot_features(model, (10,14))